#Watson submission EDA

In [ ]:
#Review: building a Pytorch BERT model

In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 15.9 MB/s 
     |████████████████████████████████| 6.6 MB 68.5 MB/s 
     |████████████████████████████████| 163 kB 82.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 15.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 348 kB 15.8 MB/s 
     |████████████████████████████████| 209 kB 74.2 MB/s 
     |████████████████████████████████| 81 kB 9.8 MB/s 
     |████████████████████████████████| 78 kB 8.3 MB/s 
     |████████████████████████████████| 112 kB 79.0 MB/s 
     |████████████████████████████████| 49 kB 7.5 MB/s 
     |████████████████████████████████| 147 kB 90.0 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.wh

In [ ]:
!nvidia-smi

Mon Oct  3 23:55:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch.nn as nn
import transformers
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

class cfg:

  #Data
  data_loc = 'PATH TO TRAINING DATA HERE'
  fold_df_loc = '/content/data_fold.csv'
  n_folds = 5

  #Model name
  model_name = 'joeddav/xlm-roberta-large-xnli'
  
  gradient_checkpointing = False
  gradient_accumulation_steps = 1
  
  #To use f16?
  apex = False
  
  #Define parameters
  batch_size = 8
  max_epochs = 3
  learning_rate = 5e-6

  #Early stopping
  patience = 1
  max_len = 100

  #Try out another loss:
  criterion = nn.CrossEntropyLoss()

  #Scheduler
  scheduler = 'linear'
  num_cycles = 0.5
  num_warmup_steps = 0

  #dropout
  hidden_dropout = 0.1
  hidden_dropout_prob = 0.1
  attention_dropout = 0.1
  attention_dropout_prob = 0.1

  #last layer id
  last_layer_id = -1



transformers.__version__: 4.22.2


In [ ]:
import pandas as pd
from sklearn.model_selection import KFold

df = pd.read_csv(cfg.data_loc)

df.loc[:, 'kfold'] = -1
df.sample(frac=1, random_state=23).reset_index(drop=True)
labels = df.label.values

kfolds = KFold(n_splits = cfg.n_folds)

for fold, (trn,val) in enumerate(kfolds.split(X=df, y=labels)):
  df.loc[val, 'kfold'] = fold

cfg.train_len = len(df[df.kfold != 0])

df.to_csv("data_fold.csv", index=False)

#DataLoader

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np

class WatsonDataset(Dataset):
  def __init__(self, data, tokenizer, max_len, inference=False):
    
    self.len = len(data)
    self.max_len = max_len
    self.data = data
    self.tokenizer = tokenizer
    self.inference = inference

  def __getitem__(self, idx):
    
    #Store premise and hypothesis
    premis = self.data.premise[idx]
    hypoth = self.data.hypothesis[idx]

    #Combine into a single text string
    txt = premis + ' [SEP] ' + hypoth

    #Store label if not inference
    if self.inference == False:
      label = self.data.label[idx]

    #Encode with tokenizer
    encoding = self.tokenizer(txt, padding='max_length',
                              truncation=True, 
                              max_length=self.max_len)
    
    #Convert to tensor
    item = {key:torch.as_tensor(val) for key,val in encoding.items()}
    
    if self.inference == False:
      item['label'] = torch.as_tensor(label)
    
    return item

  def __len__(self):
    return self.len

# The model/Initialization

In [ ]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
        
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings
    

class PooledCustomModel(nn.Module):

    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model_name, output_hidden_states=True)
            self.config.hidden_dropout = self.cfg.hidden_dropout
            self.config.hidden_dropout_prob = self.cfg.hidden_dropout_prob
            self.config.attention_dropout = self.cfg.attention_dropout
            self.config.attention_probs_dropout_prob = self.cfg.attention_dropout_prob
            self.config.layer_norm_eps = self.cfg.layer_norm_eps
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model_name, config=self.config)
        else:
            self.model = AutoModel(self.config)
        if self.cfg.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()
        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, 3)
        self._init_weights(self.fc)
        
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
        
    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[2][self.cfg.last_layer_id]
        feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature

    def forward(self, inputs):
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output

class Learning: #Integrate option of changing loss
  def __init__(self, model, device, optimizer, cfg):
    self.model = model
    self.device = device
    self.optimizer = optimizer
    self.num_train_steps = cfg.train_len / cfg.batch_size * cfg.max_epochs

  def loss_fn(self, output, label):
    loss = nn.CrossEntropyLoss().to(self.device)
    return loss(output, label)

  def get_scheduler(self, cfg):
    if cfg.scheduler == 'linear':
        scheduler = get_linear_schedule_with_warmup(
            self.optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=self.num_train_steps
        )
    elif cfg.scheduler == 'cosine':
        scheduler = get_cosine_schedule_with_warmup(
            self.optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=self.num_train_steps, num_cycles=cfg.num_cycles
        )
    return scheduler

  def train_fn(self, dataloader):

    self.model.train()
    scheduler = self.get_scheduler(cfg)
    scaler = torch.cuda.amp.GradScaler(enabled=cfg.apex)

    #tracking
    step_loss = 0
    training_steps = 0
    pred_list, lab_list = [], []

    for data in dataloader:

      #To device
      input_ids = data['input_ids'].to(self.device)
      masks = data['attention_mask'].to(self.device)
      labels = data['label'].to(self.device)

      loss_fn = self.loss_fn

      #Calc logits and loss
      with torch.cuda.amp.autocast(enabled=cfg.apex):
        logits = self.model({'input_ids':input_ids, 'attention_mask':masks})
        loss = loss_fn(logits, labels)
      
      #Record step_loss and loss
      step_loss += loss.item()
      training_steps += 1
      if training_steps % 200 == 0:
        print(f"Training loss after {training_steps} training steps: {step_loss/(training_steps*cfg.batch_size)}")

      #Flatten/take argmax of logits
      flatten_labs = labels.view(-1).cpu().numpy()
      training_logits = logits.view(-1, 3)
      predictions = torch.argmax(training_logits, axis=1).cpu().numpy()

      #For accuracy calc
      pred_list.extend(predictions)
      lab_list.extend(flatten_labs)

      #gradient clipping
      torch.nn.utils.clip_grad_norm_(parameters=self.model.parameters(), max_norm=10)

      #Backward/optimizer/scheduler
      self.optimizer.zero_grad()
      
      if cfg.apex == True:
        scaler.scale(loss).backward()
        scaler.step(self.optimizer)
        scaler.update()

        self.optimizer.zero_grad()
        scheduler.step()
      
      else:
        loss.backward()
        self.optimizer.step()
        scheduler.step()

    #Calc acc
    train_acc = accuracy_score(lab_list, pred_list)
    train_loss = step_loss/training_steps
    return train_acc, train_loss

  def valid_fn(self, dataloader):

    self.model.eval()

    lab_list, pred_list = [], []
    val_loss = 0

    for data in dataloader:

      input_ids = data['input_ids'].to(self.device)
      masks = data['attention_mask'].to(self.device)
      labels = data['label'].to(self.device)

      logits = self.model({'input_ids':input_ids, 'attention_mask':masks})
      loss = self.loss_fn(logits, labels)
      val_loss += loss.item()

      lab_list.extend(labels.view(-1).cpu().numpy())
      pred_list.extend(torch.argmax(logits, axis=-1).cpu().numpy())

    val_acc = accuracy_score(lab_list, pred_list)
    val_loss = val_loss/len(lab_list)

    return val_acc, val_loss

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import time
import gc
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def run_experiment(fold, cfg, params, save_model=False):

  print(params)

  #Load model and tokenizer
  tokenizer = AutoTokenizer.from_pretrained(cfg.model_name, fast=True)

  #pull in df:
  df = pd.read_csv(cfg.fold_df_loc)

  #Pull out on the split
  train_df = df[df.kfold != fold].reset_index(drop=True)
  valid_df = df[df.kfold == fold].reset_index(drop=True)

  #Create datasets
  train_data = WatsonDataset(train_df, tokenizer, cfg.max_len)
  valid_data = WatsonDataset(valid_df, tokenizer, cfg.max_len)

  #Feed to dataloader
  train_loader = DataLoader(train_data, batch_size=cfg.batch_size, shuffle=True,
                                num_workers=2, pin_memory=True)
  valid_loader = DataLoader(valid_data, batch_size=cfg.batch_size, shuffle=True,
                                num_workers=2, pin_memory=True)


  #UPDATE CFG:
  cfg.num_training_steps = len(train_df)/cfg.batch_size*cfg.max_epochs

  cfg.attention_dropout = params['attention_dropout']
  cfg.attention_dropout_prob = params['attention_dropout_prob']
  cfg.hidden_dropout = params['hidden_dropout']
  cfg.hidden_dropout_prob = params['hidden_dropout_prob']
  cfg.last_layer_id = params['last_layer_id']
  cfg.learning_rate = params['learning_rate']
  cfg.layer_norm_eps = params['layer_norm_eps']


  #Calculate number of training steps: cfg
  model = PooledCustomModel(cfg, pretrained=True)
  model = model.to(device)

  #Select optimizer
  optimizer = torch.optim.AdamW(model.parameters(), lr=cfg.learning_rate)

  #Instantiate for training loop
  lrn = Learning(model, device, optimizer, cfg)

  best_acc = 0
  best_epoch = 0
  patience = 1
  no_imp = 0

  val_acc = []
  val_loss = []

  for epoch in range(cfg.max_epochs):
    train_acc, train_loss = lrn.train_fn(train_loader)
    valid_acc, valid_loss = lrn.valid_fn(valid_loader)
    
    val_acc.append(valid_acc)
    val_loss.append(valid_loss)

    print(f'validation acc for epoch {epoch}: {valid_acc}')
    if valid_acc > best_acc:
      best_acc = valid_acc
      best_loss = valid_loss
      best_epoch = epoch
    
    else:
      no_imp += 1
      if no_imp >= patience:
        break

  #Clean up
  model = None
  torch.cuda.empty_cache()
  gc.collect()

  return best_acc, best_loss, best_epoch

In [ ]:
import optuna

def objective(trial):
  params = {
      "layer_norm_eps": trial.suggest_float("layer_norm_eps", 1e-8, 1e-4),
      "attention_dropout": trial.suggest_float("attention_dropout", .1, .5),
      "attention_dropout_prob": trial.suggest_float("attention_dropout_prob", .1, .5),
      "hidden_dropout": trial.suggest_float("hidden_dropout", .1, .5),
      "hidden_dropout_prob": trial.suggest_float("hidden_dropout_prob", .1, .5),
      "last_layer_id": trial.suggest_int("last_layer_id", -3, -1),
      "learning_rate": trial.suggest_float("learning_rate", 5e-6, 1e-5)
      }

  all_losses = []

  for f_ in range(cfg.n_folds):
    best_acc, best_loss, best_epoch = run_experiment(f_, cfg, params)
    all_losses.append(best_loss)

  return np.mean(all_losses)

In [ ]:
study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials=20)
print("Best Trial:")
trial_ = study.best_trial
print(trial_.values)

[I 2022-10-03 23:54:46,008] A new study created in memory with name: no-name-a20b1ba9-638d-4799-b831-710126825a52


{'layer_norm_eps': 3.037430959850435e-05, 'attention_dropout': 0.3813872637346437, 'attention_dropout_prob': 0.32310615290183314, 'hidden_dropout': 0.36452341638266383, 'hidden_dropout_prob': 0.1219189099004625, 'last_layer_id': -3, 'learning_rate': 7.768328378877015e-06}


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaModel: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Training loss after 200 training steps: 0.05240012298570946
Training loss after 400 training steps: 0.04565173692069948
Training loss after 600 training steps: 0.04373088080358381
Training loss after 800 training steps: 0.043266596817120445
Training loss after 1000 training steps: 0.042002600682899356
Training loss after 1200 training steps: 0.041971399336859276
validation acc for epoch 0: 0.9253300330033003
Training loss after 200 training steps: 0.026895898627117276
Training loss after 400 training steps: 0.029071907158649993
Training loss after 600 training steps: 0.028707180234778206
Training loss after 800 training steps: 0.028923143827123566
Training loss after 1000 training steps: 0.029978673623350913
Training loss after 1200 training steps: 0.029940540409346187
validation acc for epoch 1: 0.9141914191419142
{'layer_norm_eps': 3.037430959850435e-05, 'attention_dropout': 0.3813872637346437, 'attention_dropout_prob': 0.32310615290183314, 'hidden_dropout': 0.36452341638266383, 'hid

Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaModel: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

Training loss after 200 training steps: 0.09948994142003358
Training loss after 400 training steps: 0.09061214630492032
Training loss after 600 training steps: 0.08815529856830835
Training loss after 800 training steps: 0.08525891138473525
Training loss after 1000 training steps: 0.08347475728206337
Training loss after 1200 training steps: 0.08117030708429714
validation acc for epoch 0: 0.9055280528052805
Training loss after 200 training steps: 0.06649654109496624
Training loss after 400 training steps: 0.06938131520524621
Training loss after 600 training steps: 0.06801595942505324
Training loss after 800 training steps: 0.06712832045159303
Training loss after 1000 training steps: 0.06692503215558827
Training loss after 1200 training steps: 0.06655393493594602


[I 2022-10-04 21:09:52,297] Trial 6 finished with value: 0.03516959551634981 and parameters: {'layer_norm_eps': 8.062777030495783e-05, 'attention_dropout': 0.4855381495921697, 'attention_dropout_prob': 0.4467636640833267, 'hidden_dropout': 0.22779719425671316, 'hidden_dropout_prob': 0.2652048337141848, 'last_layer_id': -2, 'learning_rate': 7.147419945425295e-06}. Best is trial 5 with value: 0.02980479276846639.


validation acc for epoch 1: 0.9042904290429042
{'layer_norm_eps': 7.608310991151871e-05, 'attention_dropout': 0.2808706151117537, 'attention_dropout_prob': 0.13711302895034547, 'hidden_dropout': 0.33438628848911534, 'hidden_dropout_prob': 0.41812717114401143, 'last_layer_id': -1, 'learning_rate': 5.162688235743372e-06}


Some weights of the model checkpoint at joeddav/xlm-roberta-large-xnli were not used when initializing XLMRobertaModel: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Training loss after 200 training steps: 0.11043811371549964
Training loss after 400 training steps: 0.10311910153366625
Training loss after 600 training steps: 0.0994136209289233
Training loss after 800 training steps: 0.09678489528829232
Training loss after 1000 training steps: 0.09503364639729261
Training loss after 1200 training steps: 0.09335213714589675
